In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.models import *
from keras import backend as K
from keras.layers import Input, Flatten, Dense, Reshape, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization, Activation, Lambda,Subtract

In [2]:
%tensorflow_version 2.x
%matplotlib inline

## Loading in our Data

In [3]:
#colab-related
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
X_train = np.load('/content/gdrive/My Drive/613/X_train.npy', allow_pickle=True)/255
noisy_X_train = np.load('/content/gdrive/My Drive/613/noisy_X_train.npy', allow_pickle=True)/255

X_val = np.load('/content/gdrive/My Drive/613/X_val.npy', allow_pickle=True)/255
noisy_X_val = np.load('/content/gdrive/My Drive/613/noisy_X_val.npy', allow_pickle=True)/255

X_test = np.load('/content/gdrive/My Drive/613/X_test.npy', allow_pickle=True)/255
noisy_X_test = np.load('/content/gdrive/My Drive/613/noisy_X_test.npy', allow_pickle=True)/255

## Model

In [5]:
inpt = Input(shape=(None,None,1))
# 1st layer, Conv+relu
x = Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='same')(inpt)
x = Activation('relu')(x)
# 15 layers, Conv+BN+relu
for i in range(5):
    x = Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)   
# last layer, Conv
x = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), padding='same')(x)
x = Subtract()([inpt, x])   # input - noise
model = Model(inputs=inpt, outputs=x)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   160         ['input_1[0][0]']                
                                16)                                                               
                                                                                                  
 activation (Activation)        (None, None, None,   0           ['conv2d[0][0]']                 
                                16)                                                           

## Train

In [29]:
def ssim_loss(y_true, y_pred):
  return tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0, filter_size=4))

In [30]:
from keras.callbacks import ModelCheckpoint
model.compile(optimizer="adam", loss=['mse'], metrics = ssim_loss)

# use call back functions
ckpt = ModelCheckpoint('/model_{epoch:02d}.h5', monitor='val_loss', 
                verbose=0, save_best_only=True)
history = model.fit(noisy_X_train, X_train, epochs=30, batch_size=32, shuffle=True, validation_data=(noisy_X_val, X_val), callbacks=[ckpt])

Epoch 1/30
1407/1407 [==============================] - 13s 8ms/step - loss: 0.0023 - ssim_loss: 0.9260 - val_loss: 0.0379 - val_ssim_loss: 0.5947
Epoch 2/30
1407/1407 [==============================] - 12s 8ms/step - loss: 0.0022 - ssim_loss: 0.9312 - val_loss: 0.0066 - val_ssim_loss: 0.8214
Epoch 3/30
1407/1407 [==============================] - 11s 8ms/step - loss: 0.0022 - ssim_loss: 0.9342 - val_loss: 0.0054 - val_ssim_loss: 0.8375
Epoch 4/30
1407/1407 [==============================] - 12s 8ms/step - loss: 0.0021 - ssim_loss: 0.9373 - val_loss: 0.0292 - val_ssim_loss: 0.6922
Epoch 5/30
1407/1407 [==============================] - 11s 8ms/step - loss: 0.0021 - ssim_loss: 0.9384 - val_loss: 0.0268 - val_ssim_loss: 0.6748
Epoch 6/30
1407/1407 [==============================] - 11s 8ms/step - loss: 0.0021 - ssim_loss: 0.9405 - val_loss: 0.0052 - val_ssim_loss: 0.8979
Epoch 7/30
1407/1407 [==============================] - 12s 8ms/step - loss: 0.0020 - ssim_loss: 0.9438 - val_loss: 0.

In [31]:
model.evaluate(noisy_X_test, X_test, batch_size = 32)

313/313 [==============================] - 1s 4ms/step - loss: 0.0070 - ssim_loss: 0.8967


[0.00696578761562705, 0.8966607451438904]

In [32]:
y_pred = model.predict(noisy_X_test)

In [33]:
from skimage.metrics import structural_similarity as ssim

t = 0
for i in range(10000):
  t += ssim(X_test[i,:,:], y_pred[i,:,:,:].reshape(28,28));
t = t/10000
t

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  """


0.8889254960595064

In [34]:
X_test.reshape((-1, 28, 28, 1)).shape

(10000, 28, 28, 1)

In [35]:
skimage.metrics.peak_signal_noise_ratio(X_test.reshape((-1, 28, 28, 1)), y_pred, data_range=1)

21.57029815836356